In [ ]:
import os
import os.path
import random
import string

import cherrypy

from app.web import TemplateHtml

class StringGenerator(TemplateHtml):
    def __init__(self, template_path, config_path):
        super().__init__(template_path, config_path)
        
    @cherrypy.expose
    def index(self):
        yield self.encode('utf-8')


@cherrypy.expose
class StringGeneratorWebService:

    @cherrypy.tools.accept(media='text/plain')
    def GET(self):
        return cherrypy.session.get('mystring')

    def POST(self, length=8):
        some_string = ''.join(random.sample(string.hexdigits, int(length)))
        cherrypy.session['mystring'] = some_string
        return some_string

    def PUT(self, another_string):
        cherrypy.session['mystring'] = another_string

    def DELETE(self):
        cherrypy.session.pop('mystring', None)


if __name__ == '__main__':
    conf = {
        '/': {
            'tools.sessions.on': True,
            'tools.staticdir.root': os.path.abspath(os.getcwd())
        },
        '/generator': {
            'request.dispatch': cherrypy.dispatch.MethodDispatcher(),
            'tools.response_headers.on': True,
            'tools.response_headers.headers': [('Content-Type', 'text/plain')],
        },
        '/static': {
            'tools.staticdir.on': True,
            'tools.staticdir.dir': './public'
        }
    }
    webapp = StringGenerator()
    webapp.generator = StringGeneratorWebService()
    cherrypy.quickstart(webapp, '/', conf)

In [ ]:
import random
import string
from io import BytesIO

from base64 import b64encode
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
matplotlib.use('Agg')
import cherrypy

from app.web import TemplateHtml

class Html(TemplateHtml):
    def __init__(self, template_path, config_path):
        super().__init__(template_path, config_path)
        
    @cherrypy.expose
    def display2(self):
        # Modified from https://gist.github.com/tkf/1299670 by @tkf
        d = self.plot()
        d = d.decode('utf8')
        return f'''<img src="data:image/png;base64,{d}" width="640" height="480" border="0">'''

    def plot(self):
        image = BytesIO()
        x = np.linspace(0, 10)
        y = np.sin(x)
        plt.plot(x, y)
        plt.savefig(image, format='png')
        return b64encode(image.getvalue())

    @cherrypy.expose
    def index(self):
        yield self.encode('utf-8')

    @cherrypy.expose
    def generate(self, length=8):
        some_string = ''.join(random.sample(string.hexdigits, int(length)))
        cherrypy.session['mystring'] = some_string
        yield some_string

    @cherrypy.expose
    def display(self):
        yield cherrypy.session['mystring']

template_path = 'templates/form.html'
config_path = 'configs/form.toml'
config = 'configs/base-server.conf'
httpd = Html(template_path, config_path)
httpd.quickstart(config=config)

[20/Mar/2021:16:18:54] ENGINE Listening for SIGTERM.
[20/Mar/2021:16:18:54] ENGINE Bus STARTING
[20/Mar/2021:16:18:54] ENGINE Set handler for console events.
[20/Mar/2021:16:18:54] ENGINE Started monitor thread 'Autoreloader'.
[20/Mar/2021:16:18:54] ENGINE Serving on http://192.168.100.191:9999
[20/Mar/2021:16:18:54] ENGINE Bus STARTED


192.168.100.222 - - [20/Mar/2021:16:19:01] "GET /display2 HTTP/1.1" 200 19586 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36 Edg/89.0.774.54"
192.168.100.222 - - [20/Mar/2021:16:19:01] "GET /favicon.ico HTTP/1.1" 200 1406 "http://192.168.100.191:9999/display2" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36 Edg/89.0.774.54"
192.168.100.222 - - [20/Mar/2021:16:28:56] "GET / HTTP/1.1" 200 560 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36 Edg/89.0.774.54"


[20/Mar/2021:17:01:00] ENGINE Restarting because configs/base-server.conf changed.
[20/Mar/2021:17:01:00] ENGINE Stopped thread 'Autoreloader'.
[20/Mar/2021:17:01:00] ENGINE Bus STOPPING
[20/Mar/2021:17:01:00] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('192.168.100.191', 9999)) shut down
[20/Mar/2021:17:01:00] ENGINE Removed handler for console events.
[20/Mar/2021:17:01:00] ENGINE Bus STOPPED
[20/Mar/2021:17:01:00] ENGINE Bus EXITING
[20/Mar/2021:17:01:00] ENGINE Waiting for child threads to terminate...
[20/Mar/2021:17:01:00] ENGINE Bus EXITED
[20/Mar/2021:17:01:00] ENGINE Re-spawning C:\Users\xinet\anaconda3\envs\ui\python.exe -m ipykernel_launcher -f C:\Users\xinet\AppData\Roaming\jupyter\runtime\kernel-ddc085ac-8bdd-4d62-8a03-d8efda77216d.json


In [8]:
import folium
import pandas as pd

In [16]:
# San Francisco latitude and longitude values
latitude = 30.1147
longitude = 120.211 

# Create map and display it
san_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# Display the map of San Francisco
san_map